*Technology transfer for rapid family forest assessments and stewardship planning*

## Compile Landsat Data and Calculate Vegetation Indicies

This notebook facilitates the process of collecting 17 years of Landsat multispectral data (2002-2018) and calculating several vegetation indices from these images using Google Earth Engine Python API. Indices include tasseled cap brightness, greenness, and wetness, NDVI, SAVI, and ENDVI.
Images are then exported to Google drive. 

In [2]:
import ee
import IPython.display 
import pprint
import datetime
import dateutil.parser
import ipywidgets
import numpy as np
import pandas as pd
import traitlets
import ipyleaflet
from geetools import batch

# Configure the pretty printing output.
pp = pprint.PrettyPrinter(depth=4)

In [48]:
##Initialize connection to server
ee.Initialize()

In [49]:
#specify coordinates for area of interest.
#this region covers western Oregon
aoi = ee.Geometry.Polygon([
                    [-124.6, 41.9], [-117.0, 41.9], [-117.0, 49.0],
                     [-124.6, 49.0], [-124.6, 41.9]])

In [50]:
#pull in Landsat 8 collection for 2013-2018
#filter to aoi
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filter(ee.Filter.lt('CLOUD_COVER', 5))\
        .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7'])\
        .filterBounds(aoi)

#print number of images in Landsat 8 collection
pp.pprint(l8sr.size().getInfo())

1093


In [51]:
#pull in Landsat 7 collection for 2012
#filter to aoi
l7sr = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filter(ee.Filter.lt('CLOUD_COVER', 5))\
        .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7'])\
        .filterBounds(aoi)

#print number of images in Landsat 7 collection
pp.pprint(l7sr.size().getInfo())

3504


In [52]:
#pull in Landsat 5 collection for 2002-2011
#filter to aoi
l5sr = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filter(ee.Filter.lt('CLOUD_COVER', 5))\
        .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7'])\
        .filterBounds(aoi)

#print number of images in Landsat 5 collection
pp.pprint(l5sr.size().getInfo())

4278


In [53]:
#create function to rename bands to common idenifiers for Landsat 8
def renameLand8(image):
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7',]
    new_bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']
    return image.select(bands).rename(new_bands)

#map function to landsat 8 collection
land8 = ee.ImageCollection(l8sr).map(renameLand8)

In [54]:
#create function to rename bands to common idenifiers for Landsat 7
def renameLand7(image):
    bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
    new_bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']
    return image.select(bands).rename(new_bands)

#map function to landsat 7 collection
land7 = ee.ImageCollection(l7sr).map(renameLand7)

In [55]:
#function to rename bands to common idenifiers for Landsat 5
def renameLand5(image):
    bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
    new_bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']
    return image.select(bands).rename(new_bands)

#map function to landsat 5 collection
land5 = ee.ImageCollection(l5sr).map(renameLand5)

In [56]:
#Landsat 8
#function to calculation tasseled cap indicies - brightness, greenness, wetness
#add these bands to the image
def tasseled_cap_8(image):
    blue = image.select("blue")
    green = image.select("green")
    red = image.select("red")
    nir = image.select("nir")
    swir1 = image.select("swir1")
    swir2 = image.select("swir2")
    #calculate tasseled cap transformations
    bright = ((blue.multiply(0.03029)).add(green.multiply(0.02786)).add(red.multiply(0.04733))\
                .add(nir.multiply(0.05599)).add(swir1.multiply(0.0508)).add(swir2.multiply(0.01872))).toFloat().rename('brightness')
    green = ((blue.multiply(-0.02941)).add(green.multiply(-0.00243)).add(red.multiply(-0.05424))\
                .add(nir.multiply(0.07276)).add(swir1.multiply(00.00713)).add(swir2.multiply(-0.01608))).toFloat().rename('greenness')
    wet = ((blue.multiply(0.01511)).add(green.multiply(0.01973)).add(red.multiply(00.03283))\
                .add(nir.multiply(0.03407)).add(swir1.multiply(-0.07117)).add(swir2.multiply(-0.04559))).toFloat().rename('wetness')
    
    return image.addBands(bright).addBands(green).addBands(wet)

In [57]:
#map tasseled cap function to landsat 8 collection
land8_tc = ee.ImageCollection(land8).map(tasseled_cap_8)

In [58]:
#Landsat 7
#function to calculation tasseled cap indicies - brightness, greenness, wetness
#add these bands to the image
def tasseled_cap_7(image):
    blue = image.select("blue")
    green = image.select("green")
    red = image.select("red")
    nir = image.select("nir")
    swir1 = image.select("swir1")
    swir2 = image.select("swir2")
    #calculate tasseled cap transformations
    bright = ((blue.multiply(0.03561)).add(green.multiply(0.03972)).add(red.multiply(0.03904))\
                .add(nir.multiply(0.06966)).add(swir1.multiply(0.02286)).add(swir2.multiply(0.01596))).toFloat().rename('brightness')
    green = ((blue.multiply(-0.03344)).add(green.multiply(-0.03544)).add(red.multiply(-0.02630))\
                .add(nir.multiply(0.06966)).add(swir1.multiply(-0.00242)).add(swir2.multiply(-0.01608))).toFloat().rename('greenness')
    wet = ((blue.multiply(0.02626)).add(green.multiply(0.02141)).add(red.multiply(0.00926))\
                .add(nir.multiply(0.00656)).add(swir1.multiply(-00.07629)).add(swir2.multiply(-0.05388))).toFloat().rename('wetness')

    return image.addBands(bright).addBands(green).addBands(wet)

In [59]:
#map tasseled cap function to landsat 7 collection
land7_tc = ee.ImageCollection(land7).map(tasseled_cap_7)

In [60]:
#Landsat 5
#function to calculation tasseled cap indicies - brightness, greenness, wetness
#add these bands to the image
def tasseled_cap_5(image):
    blue = image.select("blue")
    green = image.select("green")
    red = image.select("red")
    nir = image.select("nir")
    swir1 = image.select("swir1")
    swir2 = image.select("swir2")
    #calculate tasseled cap transformations
    bright = ((blue.multiply(0.02043)).add(green.multiply(0.04158)).add(red.multiply(0.05524))\
                .add(nir.multiply(0.05741)).add(swir1.multiply(0.03124)).add(swir2.multiply(0.02303))).toFloat().rename('brightness') 
    green = ((blue.multiply(-0.01603)).add(green.multiply(-0.02819)).add(red.multiply(-0.04934))\
                .add(nir.multiply(0.07940)).add(swir1.multiply(-0.00002)).add(swir2.multiply(-0.01446))).toFloat().rename('greenness')            
    wet = ((blue.multiply(0.00315)).add(green.multiply(0.02021)).add(red.multiply(0.03102))\
                .add(nir.multiply(0.01594)).add(swir1.multiply(-0.06806)).add(swir2.multiply(-0.06109))).toFloat().rename('wetness')
 
    return image.addBands(bright).addBands(green).addBands(wet)

In [61]:
#map tasseled cap function to landsat 5 collection
land5_tc = ee.ImageCollection(land5).map(tasseled_cap_5)

In [62]:
#merge landsat 5,7,8 collections into a single collection
land_merge = ee.ImageCollection(land5_tc.merge(land7_tc.merge(land8_tc)));

#print number of images in entire landsat collection
pp.pprint(land_merge.size().getInfo())

8875


In [63]:
#create a list of years, 2002 - 2018
years = ee.List.sequence(2002, 2018)

In [64]:
#function to create a single image for each year by taking the mean value for a given year
def make_time_series(year):
    year_filter = ee.Filter.calendarRange(year, field='year')
    month_filter = ee.Filter.calendarRange(6,9, field='month')
    filtered = land_merge.filter(year_filter).filter(month_filter)
    return filtered.mean().set('system:time_start', ee.Date.fromYMD(year, 1, 1).millis())

#map function to each year in the years list
time_series = ee.ImageCollection(years.map(make_time_series))

In [65]:
#function to add ndvi, savi, and endvi value bands to each image (year)
def indices(image):
    red = image.select('red')
    nir = image.select('nir')
    green = image.select('green')
    blue = image.select('blue')
    ndvi = (nir.subtract(red)).divide(nir.add(red)).rename('ndvi')
    savi = (nir.subtract(red).divide(nir.add(red).add(.5)).multiply(1.5)).rename('savi')
    endvi = (nir.add(green).subtract(blue.multiply(2)).divide(nir.add(green).add(blue.multiply(2)))).rename('endvi')
    #a function to compute NDVI
    return image.addBands(ndvi).addBands(savi).addBands(endvi)

In [66]:
#map function to yearly time series of landsat data
land_mets = ee.ImageCollection(time_series.map(indices))

In [67]:
#count bands in each image to see if any images are missing bands
def count(image):
    return image.set('count', image.bandNames().length())

nullimages = ee.ImageCollection(land_mets.map(count).filter(ee.Filter.eq('count', 12)))

#print number of images that have 12 bands, should equal 17 for 17 years.
pp.pprint(nullimages.size().getInfo())

17


In [68]:
#function to convert all bands to float values
#this is because all bands must have the same data type to export
def cast(image):
    return image.toFloat()

#map float function to image collection
land_mets = ee.ImageCollection(land_mets.map(cast))

In [69]:
#load layer with inventory plot buffers for BLM/USFS/DNR 
plots = ee.FeatureCollection('users/saraloreno/blm_usfs_wadnr_plot_footprints')

In [70]:
#add year as a date column to the feature collection
#calculate mean for each band within the buffer area
ft = ee.FeatureCollection(ee.List([]))

def fill(img, ini):
    inift = ee.FeatureCollection(ini)
    ft2 = img.reduceRegions(plots, ee.Reducer.mean(), scale=30)
    
    date = img.date().format('YYYY-MM-DD')
    
    ft3 = ft2.map(lambda f : f.set("date", date))
    
    return inift.merge(ft3)

plotsMean = ee.FeatureCollection(land_mets.iterate(fill, ft))

In [71]:
#export mean values to a CSV table
task = ee.batch.Export.table.toDrive(collection=plotsMean, folder='Landsat', description='plotsMean', fileFormat='CSV')
ee.batch.data.startProcessing(task.id, task.config)

{'started': 'OK'}

In [117]:
#upload boundaries for each acquisition year
area2004 = ee.FeatureCollection('users/saraloreno/2004_lidar')
area2010 = ee.FeatureCollection('users/saraloreno/2010_lidar2')
area2012 = ee.FeatureCollection('users/saraloreno/2012_lidar')
area2014 = ee.FeatureCollection('users/saraloreno/2014_lidar')
area2017 = ee.FeatureCollection('users/saraloreno/2017_lidar')

In [125]:
#filter land_merge to year of acquistion for each lidar footprint and clip to boundaries
land2004 = ee.Image(land_mets.filterDate('2004-4-01', '2004-10-31').mean()).toFloat().clip(area2004)
land2010 = ee.Image(land_mets.filterDate('2010-4-01', '2010-10-31').mean()).toFloat().clip(area2010)
land2012 = ee.Image(land_mets.filterDate('2012-4-01', '2012-10-31').mean()).toFloat().clip(area2012)
land2014 = ee.Image(land_mets.filterDate('2014-4-01', '2014-10-31').mean()).toFloat().clip(area2014)
land2017 = ee.Image(land_mets.filterDate('2017-4-01', '2017-10-31').mean()).toFloat().clip(area2017)

In [126]:
task2012 = ee.batch.Export.image.toDrive(land2012, folder='Landsat', description='land2012', scale=30,
                                    datatype="float", maxPixels = 10000000000, region=[
                    [-122.3, 42.27], [-121.87, 42.27], [-121.87, 42.00], [-122.3, 42.00], [-122.3, 42.27]])
task2012.start()

In [127]:
task2010 = ee.batch.Export.image.toDrive(land2010, folder='Landsat', description='land2010', scale=30,
                                    datatype="float", maxPixels = 10000000000, region=[
                    [-122.19, 43.07], [-120.81, 43.07], [-120.81, 42.13], [-122.19, 42.13], [-122.19, 43.07]])
task2010.start()

In [128]:
task2014 = ee.batch.Export.image.toDrive(land2014, folder='Landsat', description='land2014', scale=30,
                                    datatype="float", maxPixels = 10000000000, region=[
                    [-123.25, 45.71], [-121.79, 45.71], [-121.79, 45.2], [-123.25, 45.2], [-123.25, 45.71]])
task2014.start()

In [129]:
task2004 = ee.batch.Export.image.toDrive(land2004, folder='Landsat', description='land2004', scale=30,
                                    datatype="float", maxPixels = 10000000000, region=[
                    [-122.09, 42.76], [-120.94, 42.76], [-120.94, 42.33], [-122.09, 42.33], [-122.09, 42.76]])
task2004.start()

In [130]:
task2017 = ee.batch.Export.image.toDrive(land2017, folder='Landsat', description='land2017', scale=30,
                                    datatype="float", maxPixels = 10000000000, region=[
                    [-122.85, 42.20], [-122.31, 42.20], [-122.31, 42.00], [-122.85, 42.00], [-122.85, 42.20]])
task2017.start()

In [131]:
#export each image to Google Drive as an individual image
#colList = land_mets.toList(land_mets.size())
#colSize = colList.size().getInfo()

In [100]:
#for i in range(colSize):
#    img = ee.Image(colList.get(i))
#    imgdate = ee.Date(img.get('system:time_start')).format('yyyy-MM-dd').getInfo()
#    imgname = 'img-' + imgdate
#    ee.batch.Export.image.toDrive(img, name=imgname, scale=30, region=[[-123.6, 42.0], [-119.9, 41.9], 
#                              [-121.1, 45.6], [-123.8, 45.9], [-123.6, 42.0]], dataype='float', 
 #                                maxPixels = 10000000000).start()

In [209]:
#Display landsat 8 image
#thumbnail_url = sample_image.getThumbUrl({
#    'bands': 'wetness',
#    'min': -1,
#    'max': +1,
#    'palette': ['white', 'blue'],
#    'region': sample_image.geometry().bounds().getInfo()
#})
#IPython.display.HTML('Thumbnail URL: <a href={0}>{0}</a>'.format(thumbnail_url))
#IPython.display.Image(url=thumbnail_url)

In [210]:
#Display landsat 8 image
#thumbnail_url = sample_image.getThumbUrl({
#    'bands': 'savi',
#    'min': -1,
#    'max': 1,
#    'palette': ['blue', 'white', 'green'],
#    'region': sample_image.geometry().bounds().getInfo()
#})
#IPython.display.HTML('Thumbnail URL: <a href={0}>{0}</a>'.format(thumbnail_url))
#IPython.display.Image(url=thumbnail_url)

The following cells create a video of the landsat time series and export it to Google Drive.

In [101]:
#def image_viz(image):
#    return image.visualize({'bands': ['blue', 'green', 'red'], 
#                           'region':[[-123.6, 42.0], [-119.9, 41.9], [-121.1, 45.6], [-123.8, 45.9], [-123.6, 42.0]]})

In [102]:
#images = land_mets.map(image_viz)

In [103]:
#def convertBit(image):
#    return image.multiply(512).uint8() 
#imageVideo = images.map(convertBit)

In [105]:
#ee.batch.Export.video.toDrive(imageVideo, description='image_yearly', dimensions = 720, folder = "Landsat",
#                                 framesPerSecond = 2, region=([-123.6, 42.0], [-119.9, 41.9], [-121.1, 45.6],
#                                                              [-123.8, 45.9], [-123.6, 42.0]), 
#                                 maxFrames=10000).start()